In [1]:
import os
import sys

# # Get the path of the current notebook's directory
# project_root = os.path.dirname(os.path.abspath(''))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

from io_load import load_country_product, load_product_meta, load_product_space_vectors, load_product_space_edges, load_country_year

df = load_country_product(2023)
product_meta = load_product_meta()
vectors = load_product_space_vectors()
edges = load_product_space_edges()
country_year = load_country_year()

# Join and merge
df = df.merge(product_meta, on='product_hs92_code', how='left')
df = df.merge(vectors, on='product_hs92_code', how='left')

corrupted_codes = df[df.duplicated(subset=['country_iso3_code', 'product_hs92_code'], keep=False)]['product_hs92_code'].unique()
if corrupted_codes:
    df = df[~df['product_hs92_code'].isin(corrupted_codes)]
# QA
assert df[['country_iso3_code', 'product_hs92_code']].duplicated().sum() == 0, "Country-product codes are not unique for the given year."
assert df['distance'].between(0, 1).all(), "Distance values are not all between 0 and 1."

In [4]:
from presence import add_rca_binary, add_peer_relative_presence

test = add_rca_binary(df, threshold=1)
test = add_peer_relative_presence(test)
test

,country_iso3_code,product_hs92_code,export_value,export_rca,distance,cog,pci,product_name,product_name_short,product_parent_id,show_feasibility,natural_resource,product_space_x,product_space_y,product_space_cluster_name,x_binary,share,rel_presence,abs_presence
0,AFG,101,0,0.000000,0.950,0.003,0.272,"Live horses, asses, mules or hinnies",Horses,100,True,False,1.287945,1.262717,Agriculture,0,0.000000,0.000000,-0.000096
1,AFG,102,12632,0.021094,0.923,0.001,-0.587,Live bovine animals,Bovine,100,True,False,1.428519,1.414477,Agriculture,0,0.000009,0.008061,-0.001151
2,AFG,104,58397,0.339285,0.904,-0.000,-1.514,Live sheep and goats,Sheep,100,True,False,1.931855,1.673194,Agriculture,0,0.000043,0.011727,-0.003643
3,AFG,105,0,0.000000,0.929,0.002,-0.512,"Live poultry of the following kinds Chickens, ...",Fowl,100,True,False,1.462526,0.793219,Agriculture,0,0.000000,0.000000,-0.000152
4,AFG,106,100804,0.984228,0.901,-0.000,-1.126,Other live animals,Other live animals,100,True,False,NaN,NaN,NaN,0,0.000075,0.081235,-0.000844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228961,ZMB,9703,36239,0.021806,0.904,0.435,-0.483,"Original sculptures and statuary, in any material",Sculptures,195,True,False,-2.555010,-0.101160,Industrial Chemicals and Metals,0,0.000005,0.026258,-0.000179
228962,ZMB,9704,0,0.000000,0.916,0.002,-0.177,"Postage or revenue stamps, stamp-postmarks, fi...",Postage or revenue stamps,195,False,False,NaN,NaN,NaN,0,0.000000,0.000000,-0.000101
228963,ZMB,9705,950966,1.315950,0.875,0.000,-0.417,Collections and collectors pieces,Collectors pieces,195,True,False,NaN,NaN,NaN,1,0.000126,0.204102,-0.000493
228964,ZMB,9999,2857097,0.012723,0.908,0.002,0.044,Commodities not specified according to kind,Commodities not specified according to kind,196,False,False,NaN,NaN,NaN,0,0.000380,0.014199,-0.026367
